## Closed Loop Simulation Notebook

This notebook breaks down the closed loop simulation in different jupyter notebook cells. The aim of this notebook is to ease the debugging and development of the different packages involved in the Optical Feedback Control loop.

This notebook was last run on July 7th 2022. 
Details:
- Stack release: 2021_25, Last verified to run 2022-07-07
- ts_phosim release: v2.0.4


### Required python packages

In [1]:
import argparse
import copy
import logging
import numpy as np
import os
import shutil
import subprocess
import sys
import time
import batoid
from tqdm import tqdm

from lsst.ts.batoid.CloseLoopTask import CloseLoopTask
from lsst.ts.batoid.utils.PlotUtil import plotFwhmOfIters
from lsst.daf import butler as dafButler
from lsst.ts.wep.Utility import CamType, FilterType
from lsst.ts.batoid.utils.Utility import getPhoSimPath, getAoclcOutputPath, getCamera
from lsst.ts.batoid.utils.SensorWavefrontError import SensorWavefrontError
from lsst.ts.batoid.wfsim.wfsim import SSTBuilder

from astropy.utils.data import get_pkg_data_filename
from astropy.io import fits
import matplotlib.pyplot as plt

In [1]:
import argparse
import copy
import logging
import numpy as np
import os
import shutil
import subprocess
import sys
import time
import batoid
from tqdm import tqdm

from lsst.ts.batoid.CloseLoopTask import CloseLoopTask
from lsst.ts.batoid.utils.PlotUtil import plotFwhmOfIters
from lsst.daf import butler as dafButler
from lsst.ts.wep.Utility import CamType, FilterType
from lsst.ts.batoid.utils.Utility import getPhoSimPath, getAoclcOutputPath, getCamera
from lsst.ts.batoid.utils.SensorWavefrontError import SensorWavefrontError
from lsst.ts.batoid.wfsim.wfsim import SSTBuilder

from astropy.utils.data import get_pkg_data_filename
from astropy.io import fits
import matplotlib.pyplot as plt

outs = '/sdf/home/g/gmegias/aos/perturbations/Test2/'

# Set the parser
parser = argparse.ArgumentParser(
    description="Run AOS closed-loop simulation (default is amplifier files)."
)
parser = CloseLoopTask.setDefaultParser(parser)
parser = CloseLoopTask.setImgParser(parser)

# Get the default arguments
sys.argv = ['-f']
args = parser.parse_args()

# Print default arguments
print(args)

logger = logging.getLogger()
logger.setLevel(args.log_level)

# Initialize the ClosedLoopTask
closeLoopTask = CloseLoopTask()

AOCLCOUTPUTPATH = "/sdf/home/g/gmegias/aos/ts_batoid/output/"
os.environ["AOCLCOUTPUTPATH"] = AOCLCOUTPUTPATH

args.inst = 'comcam' 
args.boresightDeg = [0.03, -0.02]
args.eimage = None
args.skyFile = '/sdf/home/g/gmegias/aos/ts_batoid/tests/testData/sky/skyComCam.txt'
args.output = outs
args.filterType = 'ref'

if os.path.exists(args.output):
    shutil.rmtree(args.output)

Namespace(inst='comcam', filterType='', rotCam=0.0, m1m3FErr=0.05, numOfProc=1, iterNum=5, output='', log_level=20, clobber=False, pipelineFile='', boresightDeg=[0, 0], skyFile='', eimage=False)


In [2]:
boresight = [0, 0]
rotCamInDeg = args.rotCam
rotAngInDeg = 0.0
useEimg = args.eimage
m1m3ForceError = args.m1m3FErr
zAngleInDeg = 27.0912
numPro = args.numOfProc
iterNum = args.iterNum
doErsDirCont = args.clobber
pathSkyFile = args.skyFile

# Check the input arguments
camType, instName = closeLoopTask.getCamTypeAndInstName(args.inst)
filterType = closeLoopTask.getFilterType(args.filterType)
baseOutputDir = closeLoopTask.checkAndCreateBaseOutputDir(args.output)

# Config feedback control loop
closeLoopTask.configOfcCalc(instName)

# Config Batoid
closeLoopTask.configBatoidCmpt(
    instName,
    filterType,
    rotAngInDeg,
    m1m3ForceError,
    numPro,
    boresight=boresight,
    zAngleInDeg=zAngleInDeg,
    seedNum=6,
)

# Set the telescope state to be the same as the OFC
dofInUm = closeLoopTask.ofcCalc.ofc_controller.aggregated_state
closeLoopTask.batoidCmpt.setDofInUm(dofInUm)

# Get the list of referenced sensor name (field positions)

# If using wavefront sensors we measure one per pair
# and the field
if camType == CamType.LsstCam:
    cornerSensorNameList = closeLoopTask.getSensorNameListOfFields(instName)
    cornerSensorIdList = closeLoopTask.getSensorIdListOfFields(instName)
    refSensorNameList = []
    refSensorIdList = []
    for name, id in zip(cornerSensorNameList, cornerSensorIdList):
        if name.endswith("SW0"):
            refSensorNameList.append(name)
            refSensorIdList.append(id)
else:
    refSensorNameList = closeLoopTask.getSensorNameListOfFields(instName)
    refSensorLocationList = closeLoopTask.getSensorLocationListOfFields(instName)
    refSensorIdList = closeLoopTask.getSensorIdListOfFields(instName)


# Common file and directory names
opdZkFileName = "opd.zer"
opdPssnFileName = "PSSN.txt"
outputDirName = "pert"
outputImgDirName = "img"
iterDefaultDirName = "iter"
dofInUmFileName = "dofPertInNextIter.mat"
fwhmItersFileName = "fwhmIters.png"
if args.pipelineFile == "":
    pipelineFile = None

# Specific file names to the amplifier/eimage
wfsZkFileName = "wfs.zer"

# Set observation Id
obsId = 9006000

In [3]:
def fwhm_compute(closeLoopTask, iterCount, iterDefaultDirName, outputDirName, 
                baseOutputDir, refSensorIdList, refSensorLocationList, obsId, 
                pdPssnFileName, opdZkFileName, rotCamInDeg): 
    # Set the observation Id
    closeLoopTask.batoidCmpt.setSurveyParam(
        obsId=obsId,
    )

    # The iteration directory
    iterDirName = "%s%d" % (iterDefaultDirName, iterCount)

    outputDir = os.path.join(baseOutputDir, iterDirName, outputDirName)
    closeLoopTask.batoidCmpt.setOutputDir(outputDir)

    outputDir = os.path.join(baseOutputDir, iterDirName, outputImgDirName)
    closeLoopTask.batoidCmpt.setOutputImgDir(outputDir)

    listOfWfErr = closeLoopTask.batoidCmpt.runBatoid(
        obsId, refSensorIdList, refSensorLocationList
    )

    closeLoopTask.batoidCmpt.analyzeOpdData(
        instName,
        zkFileName=opdZkFileName,
        rotOpdInDeg=-rotCamInDeg,
        pssnFileName=opdPssnFileName,
    )

    # Get the PSSN from file
    pssn = closeLoopTask.batoidCmpt.getOpdPssnFromFile(opdPssnFileName)

    # Get the GQ effective FWHM from file
    gqEffFwhm = closeLoopTask.batoidCmpt.getOpdGqEffFwhmFromFile(opdPssnFileName)

    # Set the FWHM data
    fwhm, sensor_id = closeLoopTask.batoidCmpt.getListOfFwhmSensorData(
        opdPssnFileName, refSensorIdList
    )

    closeLoopTask.ofcCalc.set_fwhm_data(fwhm, sensor_id)

    # Calculate the DOF
    wfe = np.array(
        [sensor_wfe.getAnnularZernikePoly() for sensor_wfe in listOfWfErr]
    )
    sensor_ids = np.array(
        [sensor_wfe.getSensorId() for sensor_wfe in listOfWfErr]
    )

    return gqEffFwhm, wfe, sensor_ids

In [4]:
gqEffFwhm_old = 0
gqEffFwhm = 1
iterCount = 0
while abs(gqEffFwhm_old - gqEffFwhm) > 1e-5:
    gqEffFwhm_old = gqEffFwhm
    gqEffFwhm, wfe, sensor_ids = fwhm_compute(closeLoopTask, iterCount, iterDefaultDirName, outputDirName, 
                baseOutputDir, refSensorIdList, refSensorLocationList, obsId, 
                opdPssnFileName, opdZkFileName, rotCamInDeg)

    print(gqEffFwhm)

    closeLoopTask.ofcCalc.calculate_corrections(
            wfe=wfe,
            field_idx=sensor_ids,
            filter_name=str('R'),
            gain=-1,
            rot=rotAngInDeg,
        )
    dofInUm = closeLoopTask.ofcCalc.ofc_controller.aggregated_state 
    dofInUm_old = copy.deepcopy(dofInUm)

    closeLoopTask.batoidCmpt.setDofInUm(dofInUm)
    
    # Save the DOF file
    closeLoopTask.batoidCmpt.saveDofInUmFileForNextIter(dofInUm)

    # Add the observation ID by 10 for the next iteration
    obsId += 10
    iterCount += 1

alphs = [0.2, 0.8, 0.4, 0.3, 0.05, 0.5, 0.05, 0.05e-01,   0.05e-02, 0.05e-02,  0.05e-02,
       0.05e-01,  0.05e-02,  0.05e-02, 0.05e-02,
        0.05e-03,  0.05e-03,  0.05e-03, 0.05e-03,
       0.05e-04, 0.05e-04,  0.05e-04,  0.05e-04,
       0.05e-04, 0.05e-04, 0.05e-05, 0.05e-04,
       0.05e-05, 0.05e-04,  0.05e-01,  0.05e-02,
        0.05e-02, 0.05e-02, 0.05e-02,  0.05e-02,
        0.05e-02,  0.05e-03, 0.05e-02, 0.05e-03,
        0.05e-03, 0.05e-05,  0.05e-03,  0.05e-04,
        0.05e-04, 0.05e-06, 0.05e-06,  0.05e-04,
       0.05e-04,  0.05e-05]


for ll in range(3):
    for idx in range(50):
        print(gqEffFwhm_old)

        for idy in range(5):
            gqEffFwhm_old, wfe, sensor_ids = fwhm_compute(closeLoopTask, iterCount, iterDefaultDirName, outputDirName, 
                        baseOutputDir, refSensorIdList, refSensorLocationList, obsId, 
                        opdPssnFileName, opdZkFileName, rotCamInDeg)


            #dofInUm[1] = np.random.normal(loc = dofInUm_old[1], scale = 0.5*np.sqrt(abs(dofInUm_old[1])))

            dofInUm[idx] = dofInUm_old[idx] + 0.5*np.sqrt(np.abs(dofInUm_old[idx]))

            closeLoopTask.batoidCmpt.setDofInUm(dofInUm)
        
            gqEffFwhm, wfe, sensor_ids = fwhm_compute(closeLoopTask, iterCount, iterDefaultDirName, outputDirName, 
                    baseOutputDir, refSensorIdList, refSensorLocationList, obsId, 
                    opdPssnFileName, opdZkFileName, rotCamInDeg)

            df = 1/abs(dofInUm_old[idx])*(gqEffFwhm - gqEffFwhm_old)/abs(dofInUm[idx] - dofInUm_old[idx])

            dofInUm[idx] =  dofInUm_old[idx] - np.sign(df)*alphs[idx]
            dofInUm_old[idx] = dofInUm[idx]

            closeLoopTask.batoidCmpt.setDofInUm(dofInUm)
            
            # Save the DOF file
            closeLoopTask.batoidCmpt.saveDofInUmFileForNextIter(dofInUm)

            # Add the observation ID by 10 for the next iteration
            obsId += 10
            iterCount += 1


0.5733849053944533
0.12858000130552488
0.12860161158023228
0.12860844773181476
0
0.12861015300897605
0.12840755092125328
0.12824419022426597
0.1281202131337435
0.12803572892263962
1
0.1279908104776573
0.1279482970489881
0.127906370391592
0.12786503449827963
0.12782429156880848
2
0.12778414230484286
0.12778112075380538
0.1277781438810162
0.12777521317778145
0.12777232735661587
3
0.1277694874081138
0.12772609880854754
0.12769569733882438
0.12772609880854754
0.12769569733882438
4
0.12772609880854754
0.12760719641351734
0.12749171115504815
0.12737967005069767
0.1272710956087903
5
0.1271660088106295
0.1271607162542847
0.1271581489557437
0.12715830625359076
0.12716118832943304
6
0.1271667951646307
0.12716568012425755
0.12716456450076152
0.12716344931880422
0.12716233464375534


In [5]:
dofInUm

array([-1.81074550e+01, -4.05030535e+00,  2.48379521e+00,  3.85986411e-01,
       -1.27934707e-01, -2.22582204e-01, -1.67755022e+00, -1.08959706e+00,
       -1.59591173e-01,  4.98021370e-02, -9.16574957e-02, -8.32777446e-02,
       -1.11823457e-01,  1.54234538e-02,  1.94815737e-02,  2.95477431e-02,
        2.53610806e-03,  5.50406180e-03,  2.23231661e-03, -1.08736465e-03,
       -9.78935866e-04, -2.58067937e-04,  7.52599132e-04,  1.65537181e-04,
       -8.68175103e-04, -2.28573292e-04, -7.98366252e-05, -2.22567575e-04,
       -1.75626300e-05, -4.98804661e-04,  5.12764973e-01,  8.70600337e-02,
        7.22219860e-02, -4.02995223e-02, -2.44621548e-02,  1.12837998e-02,
        2.43493205e-02,  1.60040483e-03, -1.53823339e-02, -1.66853485e-03,
        3.19075068e-03, -7.21409894e-05,  1.40993196e-03,  1.41025305e-04,
        3.64519993e-04, -3.27214052e-06, -1.78866838e-06,  3.63035670e-04,
       -3.06330965e-04,  8.66275874e-05])

### Summarize the FWHM and plot results

In [6]:
# Summarize the FWHM
fwhmItersFileName = "fwhmIters.png"
pssnFiles = [
    os.path.join(
        baseOutputDir,
        "%s%d" % (iterDefaultDirName, num),
        outputImgDirName,
        opdPssnFileName,
    )
    for num in range(35)
]
saveToFilePath = os.path.join(baseOutputDir, fwhmItersFileName)
plotFwhmOfIters(pssnFiles, saveToFilePath=saveToFilePath)
plt.grid()

In [3]:
# importing Image class from PIL package
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt

im = np.array(Image.open(saveToFilePath))
plt.figure(figsize=(10,10))
plt.imshow(im)
plt.axis('off')
plt.show()


NameError: name 'saveToFilePath' is not defined

In [ ]:
for idx in [0]:

    image_file = get_pkg_data_filename('/home/gmegias/jhome/aos/perturbations/test2253/iter0/img/opd_9006000_4.fits.gz')
    image_data = fits.getdata(image_file, ext=0)
    masked =np.ma.masked_where(image_data == 0, image_data)

    image_file = get_pkg_data_filename('/home/gmegias/jhome/aos/perturbations/testNe3/iter0/img/opd_9006000_4.fits.gz')
    image_data2 = fits.getdata(image_file, ext=0)
    masked2 =np.ma.masked_where(image_data2 == 0, image_data2)


    print(np.min(image_data))
    print(np.max(image_data))

    plt.figure(figsize = (10,9))
    plt.imshow(abs(masked - masked2))
    plt.colorbar()
    plt.savefig('/home/gmegias/jhome/aos/perturbations/trial/diff_imagsse0{}s.png'.format(idx))


In [22]:
fwhmItersFileName = "fwhmIters.png"
pssnFiles = [
    os.path.join(
       '/sdf/home/g/gmegias/aos/perturbations/Test2/',
        "iter%d" % (num),
        'img/',
        'PSSN.txt',
    )
    for num in range(153)
]
numOfFwhmData = 0
fwhmDataAll = np.array([])
for pssnFile in pssnFiles:

    fwhmData = np.loadtxt(pssnFile)[1, :]
    if numOfFwhmData == 0:
        numOfFwhmData = len(fwhmData)

    fwhmDataAll = np.append(fwhmDataAll, fwhmData)

numOfIter = len(pssnFiles)
reshapedFwhmData = fwhmDataAll.reshape((numOfIter, numOfFwhmData)).T

print(min(reshapedFwhmData[-1, :]))
# Plot the figure
plt.figure()
#plt.plot(reshapedFwhmData[:-1, :].T, "bx-")
plt.plot(reshapedFwhmData[-1, :], "r-", label="GQ FWHM_eff")
plt.xlabel("Iteration")
plt.ylabel("Arcsec")
plt.grid()
plt.minorticks_on()
plt.legend()
plt.ylim([0.12, 0.15])
plt.savefig('/sdf/home/g/gmegias/aos/hi2.png')

0.1261114902850277


In [23]:
print(reshapedFwhmData[-1, :])

[0.57338491 0.12858    0.12860161 0.12860845 0.13277867 0.13223128
 0.13171539 0.13123142 0.13077975 0.1279297  0.1278883  0.12784828
 0.12781005 0.12777461 0.12774419 0.1277377  0.12773151 0.1277256
 0.12771996 0.12764781 0.12773359 0.12764781 0.12773359 0.12764781
 0.128041   0.1280039  0.12795262 0.12789277 0.12782731 0.12715468
 0.12708626 0.12715468 0.12708626 0.12715468 0.12717759 0.12717669
 0.12717579 0.12717489 0.12717398 0.12715815 0.12715814 0.12715814
 0.12715813 0.12715813 0.12716007 0.12716007 0.12716007 0.12716007
 0.12716006 0.12714197 0.12714182 0.12714168 0.12714153 0.12714139
 0.12847776 0.12818324 0.12792303 0.12769738 0.1275065  0.12756668
 0.12751766 0.12747431 0.12743416 0.12739609 0.12739888 0.12739901
 0.12739938 0.12739901 0.12739938 0.12749313 0.12739888 0.12749313
 0.12739888 0.12749313 0.12808615 0.12801932 0.12795095 0.12788163
 0.12781184 0.12670649 0.12658074 0.1266095  0.12658074 0.1266095
 0.12667858 0.12667787 0.12667715 0.12667643 0.12667571 0.126664